In [5]:
import glob
import h5py
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch 
import os

In [26]:
with_aug = True

scale = 2
patch_size = 10

# scale = 3
# patch_size = 7

# scale = 4
# patch_size = 6

In [27]:
def convert_rgb_to_y(img, dim_order='hwc'):
    if dim_order == 'hwc':
        return 16. + (64.738 * img[..., 0] + 129.057 * img[..., 1] + 25.064 * img[..., 2]) / 256.
    else:
        return 16. + (64.738 * img[0] + 129.057 * img[1] + 25.064 * img[2]) / 256.

# 画像の加工
- [resize](https://ky-orihara.hatenablog.com/entry/2019/03/12/021106)(BICUBIC法などの解説がある)

In [21]:
import sys

In [28]:
def train(output_path,output_dir, images_dir ):
    os.mkdir(output_dir) if not os.path.isdir(output_dir) else None
    h5_file = h5py.File(output_dir+'/'+output_path,'w')
    
    lr_patches = []
    hr_patches = []
    
    for image_path in sorted(glob.glob('{}/*'.format(images_dir))): #ソートしながら画像ファイル名を取得
        hr = Image.open(image_path).convert('RGB') #画像ファイルをRGBに転換する
        hr_images = [] #配列を作成する
        
        if with_aug == True: #拡張したいとき
            for s in [1.0, 0.9, 0.8 ,0.7,0.6]: #0.6~1.0倍
                for r in [0, 90, 180, 270]: #0~270度回転
                    tmp = hr.resize((int(hr.width * s), int(hr.height * s)), resample=Image.BICUBIC) #拡大させる
                    tmp = tmp.rotate(r, expand=True) #回転させる
                    hr_images.append(tmp)
        else:
            hr_images.append(hr) #配列に追加する
        for hr in hr_images:# hr_imagesを1つずつ展開
#             hr_width = (hr.width // scale) * scale# 横のサイズを変更(少しだけ？)
#             hr_height = (hr.height // scale) * scale # 縦のサイズを変更
            #hr = hr.resize((hr.width, hr.height), resample=Image.BICUBIC)# hrの縦横をバイキュービック法を使用し、変更したサイズにする
            lr = hr.resize((hr.width // scale, hr.height // scale), resample=Image.BICUBIC)# scaleで割った縦横サイズでlrを作成する
            hr = np.array(hr).astype(np.float32)#npの配列にする
            lr = np.array(lr).astype(np.float32) #npの配列にする
            hr = convert_rgb_to_y(hr)#RGBからYCbCrの変換で色がめちゃくちゃにならないように、自作関数を使いHR画像の配列を変換させる
            lr = convert_rgb_to_y(lr) #同じく,自作関数を使いLR画像の配列を変換させる
 
            
            for i in range(0, lr.shape[0] - patch_size +1, scale): #　0　から　lr.shape[0]-patch_size+1　まで、　scale　間隔でiに代入する
                for j in range(0, lr.shape[1] - patch_size +1, scale): #　0　から　lr.shape[1]-patch_size+1　まで、　scale　間隔でiに代入する
                    lr_patches.append(lr[i : i+patch_size, j : j+patch_size])
                    hr_patches.append(hr[i*scale : i*scale+patch_size*scale, j*scale : j*scale+patch_size*scale])
    
    lr_patches = np.array(lr_patches) #lr_patchesをnp配列にする
    hr_patches = np.array(hr_patches) #lr_patchesをnp配列にする
    h5_file.create_dataset('lr',data=lr_patches)
    h5_file.create_dataset('hr',data=hr_patches)
    
    h5_file.close()

In [29]:
train('h5_of_anime-face-datasets_Train3.h5','test_dir' ,"/home/b20saito/CODE/PROX2/FSRCNN/create_h5_of_anime-face-datasets/train/")

In [30]:
def eval(output_path,output_dir, images_dir):
    os.mkdir(output_dir) if not os.path.isdir(output_dir) else None
    h5_file = h5py.File(output_dir+'/'+output_path,'w')
    
    lr_group = h5_file.create_group('lr') #lrというグループを作成する 辞書のように動作するkeyはlr valueは[]
    hr_group = h5_file.create_group('hr') #hrというグループを作成する
    
    for i, image_path in enumerate(sorted(glob.glob('{}/*'.format(images_dir)))):
        hr = Image.open(image_path).convert('RGB')
#         hr_width = (hr.width//scale) * scale #int型
#         hr_height = (hr.height//scale) * scale#int型

#         hr = hr.resize((hr.width,hr.height),resample=Image.BICUBIC) #上の計算結果の値をhrの値にする(64, 64) 
        lr = hr.resize((hr.width//scale,hr.height//scale),resample=Image.BICUBIC) #hrをscaleで割った物をlrの値にする (32, 32)
        hr = np.array(hr).astype(np.float32) #(64, 64, 3) 
        lr = np.array(lr).astype(np.float32) #(32, 32, 3)
        hr = convert_rgb_to_y(hr) #(64, 64) 
        lr = convert_rgb_to_y(lr) #(32, 32)
        
        lr_group.create_dataset(str(i),data=lr)
        hr_group.create_dataset(str(i),data=hr)
    
    h5_file.close()

In [31]:
eval('h5_of_anime-face-datasets_eval3.h5','test_dir' ,"/home/b20saito/CODE/PROX2/FSRCNN/create_h5_of_anime-face-datasets/eval/")

In [39]:
img = glob.glob("/home/b20saito/CODE/PROX2/FSRCNN/create_h5_of_anime-face-datasets/eval/*")

In [41]:
print(len(img))

10
